# Extreme Exploration

This notebook displays how adding a lateral zone can improve low and high-flow parts of the simulation.

In [1]:
import pickle
import potions as pt
import pandas as pd
from pandas import Series, DataFrame

In [2]:
data_path: str = "/home/andrew/Documents/Research/Projects/potions/test_data/train_test_camels_forcings.pickle"
with open(data_path, "rb") as f:
    data = pickle.load(f)

In [24]:
site_nos: list[str] = list(data.keys())
site_no: str = site_nos[350]
site_data = data[site_no]

In [25]:
train_q, train_forc = site_data["train"]
test_q, test_forc = site_data["test"]

In [26]:
polish: bool = False
hbv_res = pt.HbvModel.simple_calibration(train_forc, meas_streamflow=train_q, metric="combined", polish=polish)
lat_res = pt.HbvLateralModel.simple_calibration(train_forc, meas_streamflow=train_q, metric="combined", polish=polish)

In [27]:
hbv_df: DataFrame = hbv_res[1]["simulation"] # type: ignore
lat_df: DataFrame = lat_res[1]["simulation"] # type: ignore

In [28]:
lat_res[1]["nse"]

np.float64(0.6297730737926297)

In [29]:
low_flow = hbv_df["meas_streamflow_mmd"].quantile(0.1)
high_flow = hbv_df["meas_streamflow_mmd"].quantile(0.9)
low_mask = hbv_df["meas_streamflow_mmd"] <= low_flow
high_mask = hbv_df["meas_streamflow_mmd"] >= high_flow
low_hbv_df: DataFrame = hbv_df[low_mask]
high_hbv_df: DataFrame = hbv_df[high_mask]

In [30]:
pt.kge(high_hbv_df["sim_streamflow_mmd"], high_hbv_df["meas_streamflow_mmd"])

np.float64(0.45334037970819685)

In [34]:
hbv_val = pt.objective_high_flow(pt.nse, 0.90, hbv_df["meas_streamflow_mmd"], hbv_df["sim_streamflow_mmd"])
lat_val = pt.objective_high_flow(pt.nse, 0.90, lat_df["meas_streamflow_mmd"], lat_df["sim_streamflow_mmd"])
low_hbv_val = pt.objective_low_flow(pt.nse, 0.10, hbv_df["meas_streamflow_mmd"], hbv_df["sim_streamflow_mmd"])
low_lat_val = pt.objective_low_flow(pt.nse, 0.10, lat_df["meas_streamflow_mmd"], lat_df["sim_streamflow_mmd"])

print(f"HBV High-flow NSE: {round(hbv_val, 2)}")
print(f"Lateral High-flow NSE: {round(lat_val, 2)}")
print("_" * 25)
print(f"HBV Low-flow NSE: {round(low_hbv_val, 2)}")
print(f"Lateral Low-flow NSE: {round(low_lat_val, 2)}")

HBV High-flow NSE: -0.7
Lateral High-flow NSE: -1.74
_________________________
HBV Low-flow NSE: -0.82
Lateral Low-flow NSE: -0.08
